<font size=+4 color="Black"><center><b>Lexicon based Sentiment Analysis</b></center></font>
<font size=-1 color="Black"><center><b>*Series: All about NLP by Data Tattle </b></right></font>

<font size="+2" color="Green"><b>Please Upvote if you like the work</b></font>

### It gives motivation to a working professional (like me) to contribute more.

### About this notebook

## This notebook is a part of Series "[All about NLP](https://www.kaggle.com/datatattle/all-about-nlp)" and will cover lexicon based Models


![](https://static1.squarespace.com/static/5daddb33ee92bf44231c2fef/t/5f0387d2f6724b5987a29311/1594066902743/natural%2Blanguage%2Bprocessing%2Bin%2Bhealthcare%2B-%2Bforesee%2Bmedical.gif?format=1500w)
![](https://miro.medium.com/proxy/1*_JW1JaMpK_fVGld8pd1_JQ.gif)


### Most of the problems in actual practice are related to unsupervised learning, which means the label is absent (Positive/Negative). Majority of times companies do not find time to label it manually and hence settle for lexicon based models like Vader. Some, have built their own lexicons related to their field like Retail, News, etc. This notebook demonstrates lexicon based problem tackling.

Contents:

* [1. Importing data & libraries](#1)
* [2. Lexicon based models](#2)
* [3. VADER](#3)
* [4. Data Preprocessing](#4)
* [5. Model](#5)
* [6. TextBlob Model](#6)
* [7. Emotions in data](#7)

<a id="1"></a>
    
<font size="+2" color="indigo"><b>1. Importing data & libraries</b></font><br>

In [ ]:
# File Path
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
#Loading libraries
import numpy as np # provides a high-performance multidimensional array and tools for its manipulation
import pandas as pd # for data munging, it contains manipulation tools designed to make data analysis fast and easy
import re # Regular Expressions - useful for extracting information from text 
import nltk # Natural Language Tool Kit for symbolic and statistical natural language processing
import spacy # processing and understanding large volumes of text
import string # String module contains some constants, utility function, and classes for string manipulation
import re

# For viz
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from plotly import graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff
from collections import Counter

from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
pd.options.mode.chained_assignment = None

In [ ]:
#Loading File
df = pd.read_csv('/kaggle/input/coronavirus-tweets/Corona_tweets.csv',encoding='latin1')

In [ ]:
#Shape of dataframe
print(" Shape of training dataframe: ", df.shape)

In [ ]:
# Drop duplicates
df.drop_duplicates()
print(" Shape of dataframe after dropping duplicates: ", df.shape)

In [ ]:
#Null values

null= df.isnull().sum().sort_values(ascending=False)
total =df.shape[0]
percent_missing= (df.isnull().sum()/total).sort_values(ascending=False)

missing_data= pd.concat([null, percent_missing], axis=1, keys=['Total missing', 'Percent missing'])

missing_data.reset_index(inplace=True)
missing_data= missing_data.rename(columns= { "index": " column name"})
 
print ("Null Values in each column:\n", missing_data)


<a id="2"></a>
    
<font size="+2" color="indigo"><b>2. Lexicon based Methods</b></font><br>

## Let's discuss Vader algorithm first before preprocessing our file

#### Sentiment Analysis also known as Opinion Mining, which is a sub-field of Natural Language Processing (NLP), aims at identifying and extracting opinions through sentiments, attitude, and emotions of the writer within a given text. In general we see such text as tweets, reviews on shopping website (Amazon reviews), customer emails, social media messages and comments, surveys, etc. 
#### It has become highly important for the businesses around the globe to provide value and receive feedway so as to improve upon the product or service being offered & sentiment analysis plays an important role in this.
#### It has high implementation in the fields of marketing, sociology, economies, political science, etc. One big example is Trump's first electoral campaign where (one of the strategies) he targeted the citizens based on sentiments (twitter). 

#### There are primarily 3 ways:
#### 1) Rule-based methods
#### 2) Feature-based methods
#### 3) Embedding-based methods

#### Rule based are the methods which involve set of manually crafted rules to identify subjectivity, polarity, or the subject of an opinion. These involve techniques like Stemming, tokenization, part-of-speech tagging, parsing, Lexicons (i.e. lists of words and expressions). Generally, two lists of polarized words (e.g. negative words such as bad, worst, ugly, etc and positive words such as good, best, beautiful, etc) are prepared. The number of positive and negative words that appear in a given text are counted. If the number of positive word appearances is greater than the number of negative word appearances, the system returns a positive sentiment, and vice versa. If the numbers are even, the system will return a neutral sentiment. Methods involve
[AFINN](https://github.com/fnielsen/afinn), [Bing Liu's lexicon](https://www.cs.uic.edu/~liub/FBS/sentiment-analysis.html), [MPQA subjectivity lexicon](http://mpqa.cs.pitt.edu/lexicons/subj_lexicon/), [SentiWordNet](http://sentiwordnet.isti.cnr.it/http://), [TextBlob](https://textblob.readthedocs.io/en/dev/) & [VADER](https://github.com/cjhutto/vaderSentiment)

#### Since rule-based are generally naive in nature as don't take into account how words are combined in a sequence. Hence, feature-based methods relying on ML techniques like SVM, Decision trees, etc. are used where sentiment analysis is modeled as a classification problem. The first step in a machine learning text classifier is to transform the text extraction or text vectorization, and the classical approach has been bag-of-words or bag-of-ngrams with their frequency. More recently, new feature extraction techniques have been applied based on word embeddings (also known as word vectors). This kind of representations makes it possible for words with similar meaning to have a similar representation, which can improve the performance of classifiers.

#### Last is embedding based which involve [FastText](https://fasttext.cc/docs/en/supervised-tutorial.html) & [Flair](https://github.com/flairNLP/flair/)

<a id="3"></a>
    
<font size="+2" color="indigo"><b>3. VADER Sentiment Analysis</b></font><br>

### VADER sentiment analysis
Valence Aware Dictionary and sEntiment Reasoner is a lexicon and rule based sentiment analysis tool which works very well on social media sentiments. It is sensitive to both polarity (positive/negative) and intensity (strength) of emotion. It is available in the NLTK package and can be applied directly to unlabeled text data.

In [ ]:
!pip install vaderSentiment

In [ ]:
import vaderSentiment
# calling SentimentIntensityAnalyzer object
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

In [ ]:
# or use nltk


#import nltk
#nltk.download('vader_lexicon')
#from nltk.sentiment.vader import SentimentIntensityAnalyzer

#analyser = SentimentIntensityAnalyzer()

In [ ]:
# Using polarity scores for knowing the polarity of each text
def sentiment_analyzer_score(sentence):
    score = analyser.polarity_scores(sentence)
    print("{:-<40} {}".format(sentence, str(score)))

In [ ]:
#testing the function
tweet  = "I would love to watch the magic show again"
tweet2 = "What the hell they have made. Pathetic!"
tweet3 = " I do not know what to do"  
print (sentiment_analyzer_score(tweet))
print (sentiment_analyzer_score(tweet2))
print (sentiment_analyzer_score(tweet3))

#### Here neg (negative), neu (neutral), and pos (positive) represent the proportion of text falling under each category and this proportion will always sum up to 1 (e.g., 0.656 + 0.344 in first tweet).

#### Compound score is what reflects the overall score. It is sum of all lexicon ratings which is normalized between -1 (most extreme negative) and +1 (most extreme poistive). As per the scoring document ([link](https://github.com/cjhutto/vaderSentiment#about-the-scoring)) it is called as 'normalized, weighted composite score.'

#### The typical thresholds standardized for classifying sentences as positve, negative, neutral are: 
#### 1) positive sentiment: compound score >= 0.05
#### 2) neutral sentiment: (compound score > -0.05) and (compound score < 0.05)
#### 3) negative sentiment: compound score <= -0.05  

#### However, they can be adjusted for highly positive, positive, neutral, negative and highly negative classes as per the requirements.

#### Like other preprocessing in text analysis we are not required to remove emojis, slangs (abbreviations), emoticons, punctuations, etc. as VADER generates scores based on these. 
#### Some of the important parts are:


#### 1) Upper Case (Capitalization)
#### Use of upper case alphabets/words indicate the increase in magnitude of the sentiment. For example, I like the fact that monsoon is over. vs I LIKE the fact that monsoon is over.

In [ ]:
tweet  = "I like the fact that monsoon is over"
tweet2 = "I LIKE the fact that monsoon is over"

In [ ]:
print (sentiment_analyzer_score(tweet))
print (sentiment_analyzer_score(tweet2))

#### The compound score increases by 13% just by capitalization

#### 2) Punctuation
Use of punctuation like !, ?, etc. add to the intensity of the text.

In [ ]:
tweet  = "What is wrong with you"
tweet2  = "What is wrong with you?"
tweet3 = "What is wrong with you??"

In [ ]:
print (sentiment_analyzer_score(tweet))
print (sentiment_analyzer_score(tweet2))
print (sentiment_analyzer_score(tweet3))

#### 3) Conjuctions
Conjuctions lead to shift in polarity. The latter part (after the conjuction) acts as dominant part in defining the magnitude of polarity.

In [ ]:
tweet  = "He is good but his mother is irritating"
tweet2 = "The thai curry was bad, however pasta was delicious"
tweet3 = "The thai curry was ok and pasta was delicious"

In [ ]:
print (sentiment_analyzer_score(tweet))
print (sentiment_analyzer_score(tweet2))
print (sentiment_analyzer_score(tweet3))

#### 4) Degree modifiers
As the name suggests they intensify the degree in positive or negative manner as per the use.

In [ ]:
tweet = "Real Madrid's game play was good last night."
tweet2 = "Real Madrid's game play was extremely good last night."
tweet3 = "Real Madrid's game play was somewhat good last night."
tweet4 = "Real Madrid's game play was terrible last night."
tweet5 = "Real Madrid's game play was awfully terrible last night."

In [ ]:
print (sentiment_analyzer_score(tweet))
print (sentiment_analyzer_score(tweet2))
print (sentiment_analyzer_score(tweet3))
print (sentiment_analyzer_score(tweet4))
print (sentiment_analyzer_score(tweet5))

#### 5) Use of emoticons

In [ ]:
tweet = " What a fine day I am having today"
tweet2 = " What a fine day I am having today :-)"
tweet3 = " What a fine day I am having today :-) :-)"

In [ ]:
print (sentiment_analyzer_score(tweet))
print (sentiment_analyzer_score(tweet2))
print (sentiment_analyzer_score(tweet3))

#### 6) Use of emojis (utf-8 encoded)

In [ ]:
tweet = "I love the team and how they played last night"
tweet2 = "I love the team and how they played last night 💘"
tweet3 = "I love the team and how they played last night 😁"

In [ ]:
print (sentiment_analyzer_score(tweet))
print (sentiment_analyzer_score(tweet2))
print (sentiment_analyzer_score(tweet3))

#### 7) Slangs 

In [ ]:
tweet = "I am laughing like crazy"
tweet2 = "I am laughing like crazy lmao"
tweet3 = "I am laughing like crazy lol"

In [ ]:
print (sentiment_analyzer_score(tweet))
print (sentiment_analyzer_score(tweet2))
print (sentiment_analyzer_score(tweet3))

#### 8) Use of negations 

In [ ]:
tweet = "He wasn't very good at the play"
tweet2 = "He was not very good at the play"

In [ ]:
print (sentiment_analyzer_score(tweet))
print (sentiment_analyzer_score(tweet2))


#### 9) Slang words as modifiers

In [ ]:
tweet = "He is kinda bored"
tweet2 = "He is friggin bored"

In [ ]:
print (sentiment_analyzer_score(tweet))
print (sentiment_analyzer_score(tweet2))

<a id="4"></a>
    
<font size="+2" color="indigo"><b>4. Data Preprocessing</b></font><br>

### Tokenizer
#### We can't analyze whole sentences, we will use regex to tokenize sentences to list of words.

In [ ]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')
words_descriptions = df['text'].apply(tokenizer.tokenize)
words_descriptions.head()

#### When we split description into individual words, we have to create vocabulary and additionaly we can add new feature - description lengths

In [ ]:
all_words = [word for tokens in words_descriptions for word in tokens]
df['description_lengths']= [len(tokens) for tokens in words_descriptions]
VOCAB = sorted(list(set(all_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_words), len(VOCAB)))

In [ ]:
# Checking most common words
from collections import Counter
count_all_words = Counter(all_words)
count_all_words.most_common(100)

In [ ]:
#### 1-gram tokenizer
example = 'The quick brown fox jumps over the lazy dog.'

# remove the dots and make all words lower case
clean_example = re.sub(r'\.', '', example)
print(clean_example.split())

In [ ]:
# 2-gram tokenizer

example = 'The quick brown fox jumps over the lazy dog.'

without_first = example.split()[1:]
without_last = example.split()[:-1]

list(zip(without_last, without_first))

<a id="5"></a>
    
<font size="+2" color="indigo"><b>5. Model</b></font><br>

In [ ]:
print (sentiment_analyzer_score(tweet2))

#### Applying VADER

In [ ]:
df['scores'] = df['text'].apply(lambda review: analyser.polarity_scores(review))

df.head()

In [ ]:
df['compound']  = df['scores'].apply(lambda score_dict: score_dict['compound'])

df.head()

In [ ]:
def Sentimnt(x):
    if x>= 0.05:
        return "Positive"
    elif x<= -0.05:
        return "Negative"
    else:
        return "Neutral"
#df['Sentiment'] = df['compound'].apply(lambda c: 'positive' if c >=0.00  else 'negative')
df['Sentiment'] = df['compound'].apply(Sentimnt)


df.head()

In [ ]:
var1 = df.groupby('Sentiment').count()['text'].reset_index().sort_values(by='text',ascending=False)
sns.set_style("white")
sns.set_palette("Set2")
var1.style.background_gradient()

In [ ]:
plt.figure(figsize=(12,6))
sns.countplot(x='Sentiment',data=df)

In [ ]:
fig = go.Figure(go.Funnelarea(
    text =var1.Sentiment,
    values = var1.text,
    title = {"position": "top center", "text": "Funnel-Chart of Sentiment Distribution"}
    ))
fig.show()

#### Most common words

In [ ]:
df['temp_list'] = df['text'].apply(lambda x:str(x).split())
top = Counter([item for sublist in df['temp_list'] for item in sublist])
temp = pd.DataFrame(top.most_common(20))
temp.columns = ['Common_words','count']
#temp.style.background_gradient(cmap='Blues')

In [ ]:
fig = px.bar(temp, x="count", y="Common_words", title='Commmon Words in Selected Text', orientation='h', 
             width=700, height=700,color='Common_words')
fig.show()

In [ ]:
# Tree of the most common words
fig = px.treemap(temp, path=['Common_words'], values='count',title='Tree of Most Common Words')
fig.show()

In [ ]:
comment_words = '' 
stopwords = set(STOPWORDS) 
  
# iterate through the csv file 
for val in df.text: 
      
    # typecaste each val to string 
    val = str(val) 
  
    # split the value 
    tokens = val.split() 
      
    # Converts each token into lowercase 
    for i in range(len(tokens)): 
        tokens[i] = tokens[i].lower() 
      
    comment_words += " ".join(tokens)+" "
  
wordcloud = WordCloud(width = 800, height = 800, 
                background_color ='white', 
                stopwords = stopwords, 
                min_font_size = 10).generate(comment_words) 
  
# plot the WordCloud image                        
plt.figure(figsize = (8, 8), facecolor = None) 
plt.imshow(wordcloud) 
plt.axis("off") 
plt.tight_layout(pad = 0) 
  
plt.show() 

#### Positive Wordcloud

In [ ]:
comment_words = '' 
stopwords = set(STOPWORDS) 
  
df_positive = df[df["Sentiment"]== "Positive"] 
# iterate through the csv file 
for val in df_positive.text: 
      
    # typecaste each val to string 
    val = str(val) 
  
    # split the value 
    tokens = val.split() 
      
    # Converts each token into lowercase 
    for i in range(len(tokens)): 
        tokens[i] = tokens[i].lower() 
      
    comment_words += " ".join(tokens)+" "
  
wordcloud = WordCloud(width = 800, height = 800, 
                background_color ='white', 
                stopwords = stopwords, 
                min_font_size = 10).generate(comment_words) 
  
# plot the WordCloud image                        
plt.figure(figsize = (8, 8), facecolor = "green") 
plt.imshow(wordcloud) 
plt.axis("off") 
plt.tight_layout(pad = 0) 
  
plt.show() 

In [ ]:
comment_words = '' 
stopwords = set(STOPWORDS) 
  
df_negative = df[df["Sentiment"]== "Negative"] 
# iterate through the csv file 
for val in df_negative.text: 
      
    # typecaste each val to string 
    val = str(val) 
  
    # split the value 
    tokens = val.split() 
      
    # Converts each token into lowercase 
    for i in range(len(tokens)): 
        tokens[i] = tokens[i].lower() 
      
    comment_words += " ".join(tokens)+" "
  
wordcloud = WordCloud(width = 800, height = 800, 
                background_color ='white', 
                stopwords = stopwords, 
                min_font_size = 10).generate(comment_words) 
  
# plot the WordCloud image                        
plt.figure(figsize = (8, 8), facecolor = "red") 
plt.imshow(wordcloud) 
plt.axis("off") 
plt.tight_layout(pad = 0) 
  
plt.show() 

In [ ]:
comment_words = '' 
stopwords = set(STOPWORDS) 
  
df_neutral = df[df["Sentiment"]== "Neutral"] 
# iterate through the csv file 
for val in df_positive.text: 
      
    # typecaste each val to string 
    val = str(val) 
  
    # split the value 
    tokens = val.split() 
      
    # Converts each token into lowercase 
    for i in range(len(tokens)): 
        tokens[i] = tokens[i].lower() 
      
    comment_words += " ".join(tokens)+" "
  
wordcloud = WordCloud(width = 800, height = 800, 
                background_color ='white', 
                stopwords = stopwords, 
                min_font_size = 10).generate(comment_words) 
  
# plot the WordCloud image                        
plt.figure(figsize = (8, 8), facecolor = "yellow") 
plt.imshow(wordcloud) 
plt.axis("off") 
plt.tight_layout(pad = 0) 
  
plt.show() 

In [ ]:
del df_neutral
del df_positive
del df_negative

<a id="6"></a>
    
<font size="+2" color="indigo"><b>6. TextBlob Model</b></font><br>

In [ ]:
import warnings
warnings.filterwarnings('ignore')
from textblob import TextBlob, Word, Blobber

In [ ]:
tweet = "I would love to watch the magic show again"
TextBlob(tweet).sentiment 

#### The above output shows that the polarity of the sentence is 0.7, indicating that the sentiment is positive. Polarity is of 'float' type and lies in the range of -1,1, where 1 means a high positive sentiment, and -1 means a high negative sentiment.

#### The output also prints subjectivity of the text which is 0.825 in our example. Subjectivity is also of 'float' type and lies in the range of 0,1. The value closer to 1 indicates that the sentence is mostly a public opinion and not a factual piece of information and vice versa. 

In [ ]:
# Applying on dataset
df['TB_score']= df.text.apply(lambda x: TextBlob(x).sentiment)
df.head()

In [ ]:
df['TB_sentiment'] = df['text'].apply(lambda x: TextBlob(x).sentiment[0])
df.head()

<a id="7"></a>
    
<font size="+2" color="indigo"><b>7. Emotions in text</b></font><br>

### Getting emotions using NCR Emotion lexicon
[Link](https://pypi.org/project/NRCLex/)

In [ ]:
!pip install nrclex

In [ ]:
from nrclex import NRCLex

In [ ]:
tweet = NRCLex('Good work to the team')
#Return affect dictionary
print(tweet.affect_dict)
#Return raw emotional counts
print("\n",tweet.raw_emotion_scores)
#Return highest emotions
print("\n", tweet.top_emotions)
#Return affect frequencies
print("\n",tweet.affect_frequencies)

In [ ]:
text = NRCLex("Congratulations ")
# Getting top emotions
print("\n", text.top_emotions)
# Getting the top most emotion
print("\n", text.top_emotions[0][0])
# Getting the top most emotion score
print("\n", text.top_emotions[0][1])

In [ ]:
text = NRCLex("We can do it ")
# Getting top emotions
print("\n", text.top_emotions)
# Getting the top most emotion
print("\n", text.top_emotions[0][0])
# Getting the top most emotion score
print("\n", text.top_emotions[0][1])

In [ ]:
def emotion(x):
    text = NRCLex(x)
    if text.top_emotions[0][1] == 0.0:
        return "No emotion"
    else:
        return text.top_emotions[0][0]
df['Emotion'] = df['text'].apply(emotion)
df.head()

In [ ]:

import matplotlib.pyplot as plt
from matplotlib import cm
from math import log10

df_chart = df[df.Emotion != "No emotion"]
labels = df_chart.Emotion.value_counts().index.tolist()
data = df_chart.Emotion.value_counts()
#number of data points
n = len(data)
#find max value for full ring
k = 10 ** int(log10(max(data)))
m = k * (1 + max(data) // k)

#radius of donut chart
r = 1.5
#calculate width of each ring
w = r / n 

#create colors along a chosen colormap
colors = [cm.terrain(i / n) for i in range(n)]

#create figure, axis
fig, ax = plt.subplots()
ax.axis("equal")

#create rings of donut chart
for i in range(n):
    #hide labels in segments with textprops: alpha = 0 - transparent, alpha = 1 - visible
    innerring, _ = ax.pie([m - data[i], data[i]], radius = r - i * w, startangle = 90, labels = ["", labels[i]], labeldistance = 1 - 1 / (1.5 * (n - i)), textprops = {"alpha": 0}, colors = ["white", colors[i]])
    plt.setp(innerring, width = w, edgecolor = "white")

plt.legend()
plt.show()

In [ ]:
b = df_chart.Emotion.value_counts().index.tolist()
a = df_chart.Emotion.value_counts(normalize = True).tolist()
row = pd.DataFrame({'scenario' : []})
row["scenario"] = b
row["Percentage"] = a
fig = px.treemap(row, path= ["scenario"], values="Percentage",title='Tree of Emotions')
fig.show()

There are many other Emotion Lexicons like [EmoWordNet](https://www.aclweb.org/anthology/S18-2009.pdf), [EmoTxt](https://arxiv.org/ftp/arxiv/papers/1708/1708.03892.pdf) & [DepecheMood++](https://arxiv.org/pdf/1810.03660v1.pdf) which can be looked into.


## Future work: 
   1) Working on other (richer) emotion lexicons \
   2) Lexicon based models vs ML models vs Deep Learning Models 


<font size="+3" color="Green"><b>Related Work:</b></font>


### Links: Click [here](https://www.kaggle.com/datatattle/all-about-nlp) for related notebooks

<font size="+2" color="Green"><b>Please Upvote if you like the work</b></font>